In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics

import os
os.chdir('/home/aremirata/repos/7mw/')

from savvyworkout import lstm_preprocess
from savvyworkout import triaxial_lstm
from savvyworkout import triaxial_deepconvlstm
from savvyworkout import deeplearning_utils
from sklearn.externals import joblib

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

from IPython.core.display import display, HTML

%matplotlib inline

Using TensorFlow backend.


In [24]:
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Embedding, LSTM, Masking, TimeDistributed, MaxPooling1D, Flatten
from keras.layers.wrappers import TimeDistributed, Bidirectional
from keras.layers import LSTM
from keras import optimizers
from keras.preprocessing import sequence
from keras import backend as K
import keras.backend.tensorflow_backend as KTF
from keras.utils import np_utils
from sklearn.externals import joblib
from keras.callbacks import History 

In [3]:
os.chdir('/home/aremirata/repos/LSTM-Human-Activity-Recognition')

# 1 - Dataset Information

The experiments have been carried out with a group of 30 volunteers within an age bracket of 19-48 years. Each person performed six activities (WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING) wearing a smartphone (Samsung Galaxy S II) on the waist. Using its embedded accelerometer and gyroscope,3-axial linear acceleration and 3-axial angular velocity at a constant rate of 50Hz were captured. The experiments have been video-recorded to label the data manually. The obtained dataset has been randomly partitioned into two sets, where 70% of the volunteers was selected for generating the training data and 30% the test data. 

The sensor signals (accelerometer and gyroscope) were pre-processed by applying noise filters and then sampled in fixed-width sliding windows of 2.56 sec and 50% overlap (128 readings/window). The sensor acceleration signal, which has gravitational and body motion components, was separated using a Butterworth low-pass filter into body acceleration and gravity. The gravitational force is assumed to have only low frequency components, therefore a filter with 0.3 Hz cutoff frequency was used.

Discussion on what is LSTM is found on this notebook: 

### Generating Data

In [4]:
# Note: Linux bash commands start with a "!" inside those "ipython notebook" cells

DATA_PATH = "data/"

!pwd && ls
os.chdir(DATA_PATH)
!pwd && ls

!python download_dataset.py

!pwd && ls
os.chdir("..")
!pwd && ls

DATASET_PATH = DATA_PATH + "UCI HAR Dataset/"
print("\n" + "Dataset is now located at: " + DATASET_PATH)

/home/aremirata/repos/LSTM-Human-Activity-Recognition
data	 LSTM_files  lstm.py	UCI ActivityRecognition.ipynb
LICENSE  LSTM.ipynb  README.md
/home/aremirata/repos/LSTM-Human-Activity-Recognition/data
download_dataset.py  __MACOSX  source.txt  UCI HAR Dataset  UCI HAR Dataset.zip

Downloading...
Dataset already downloaded. Did not download twice.

Extracting...
Dataset already extracted. Did not extract twice.

/home/aremirata/repos/LSTM-Human-Activity-Recognition/data
download_dataset.py  __MACOSX  source.txt  UCI HAR Dataset  UCI HAR Dataset.zip
/home/aremirata/repos/LSTM-Human-Activity-Recognition
data	 LSTM_files  lstm.py	UCI ActivityRecognition.ipynb
LICENSE  LSTM.ipynb  README.md

Dataset is now located at: data/UCI HAR Dataset/


In [5]:
# Useful Constants

# Those are separate normalised input features for the neural network
INPUT_SIGNAL_TYPES = [
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

# Output classes to learn how to classify
LABELS = [
    "WALKING", 
    "WALKING_UPSTAIRS", 
    "WALKING_DOWNSTAIRS", 
    "SITTING", 
    "STANDING", 
    "LAYING"
]

In [6]:
TRAIN = "train/"
TEST = "test/"


# Load "X" (the neural network's training and testing inputs)

def load_X(X_signals_paths):
    X_signals = []
    
    for signal_type_path in X_signals_paths:
        file = open(signal_type_path, 'r')
        # Read dataset from disk, dealing with text files' syntax
        X_signals.append(
            [np.array(serie, dtype=np.float32) for serie in [
                row.replace('  ', ' ').strip().split(' ') for row in file
            ]]
        )
        file.close()
    
    return np.transpose(np.array(X_signals), (1, 2, 0))

X_train_signals_paths = [
    DATASET_PATH + TRAIN + "Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
]
X_test_signals_paths = [
    DATASET_PATH + TEST + "Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]

X_train = load_X(X_train_signals_paths)
X_test = load_X(X_test_signals_paths)


# Load "y" (the neural network's training and testing outputs)

def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]], 
        dtype=np.int32
    )
    file.close()
    
    # Substract 1 to each output class for friendly 0-based indexing 
    return y_ - 1

y_train_path = DATASET_PATH + TRAIN + "y_train.txt"
y_test_path = DATASET_PATH + TEST + "y_test.txt"

y_train = load_y(y_train_path)
y_test = load_y(y_test_path)

In [7]:
X_train.shape

(7352, 128, 3)

In [8]:
y_train.shape

(7352, 1)

In [9]:
X_test.shape

(2947, 128, 3)

In [10]:
y_test.shape

(2947, 1)

In [11]:
training_data = np.linalg.norm(X_train, axis=-1)
testing_data = np.linalg.norm(X_test, axis=-1)

In [12]:
y_train = y_train.flatten()
y_test = y_test.flatten()
x_test = testing_data

In [13]:
x_train, x_val, y_train, y_val = train_test_split(training_data, y_train, 
                                                    test_size=0.33, random_state=42)

In [14]:
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

class custom_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]
    def on_train_begin(self, logs={}):
        return
    def on_train_end(self, logs={}):
        return
    def on_epoch_begin(self, epoch, logs={}):
        return
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
        logs["roc-auc"] = roc
        logs["roc-auc_val"] = roc_val
        return
    def on_batch_begin(self, batch, logs={}):
        return
    def on_batch_end(self, batch, logs={}):
        return

In [15]:
len(np.unique(y_train)), len(np.unique(y_test)), len(np.unique(y_val))

(6, 6, 6)

In [19]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, 6)
y_test = keras.utils.to_categorical(y_test, 6)
y_val = keras.utils.to_categorical(y_val, 6)

In [20]:
x_train = sequence.pad_sequences(x_train, maxlen=128)
x_test = sequence.pad_sequences(x_test, maxlen=128)
x_val = sequence.pad_sequences(x_val, maxlen=128)

In [21]:
batch_size = 32
num_classes = 6
epochs = 20
data_augmentation = True
num_predictions = 10

In [27]:
model = Sequential()
model.add(Embedding(128, 100))
model.add(LSTM(output_dim=10,
                       init='uniform',
                       inner_init='uniform',
                       forget_bias_init='one',
                       activation='tanh',
                       inner_activation='sigmoid',
                       input_shape=x_train.shape[1:],
                       return_sequences=True))
model.add(LSTM(output_dim=10,
               init='uniform',
               inner_init='uniform',
               forget_bias_init='one',
               activation='tanh',
               inner_activation='sigmoid',
               return_sequences=True))
model.add(LSTM(output_dim=10,
                           init='uniform',
                           inner_init='uniform',
                           forget_bias_init='one',
                           activation='tanh',
                           inner_activation='sigmoid'))
model.add(Dense(num_classes, activation='softmax'))


/home/aremirata/anaconda3/envs/t2d_pred/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(kernel_initializer="uniform", recurrent_initializer="uniform", recurrent_activation="sigmoid", input_shape=(128,), return_sequences=True, units=10, unit_forget_bias=True, activation="tanh")`
  # Remove the CWD from sys.path while we load stuff.
/home/aremirata/anaconda3/envs/t2d_pred/lib/python3.5/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(unit_forget_bias=True, recurrent_initializer="uniform", kernel_initializer="uniform", recurrent_activation="sigmoid", units=10, return_sequences=True, activation="tanh")`
/home/aremirata/anaconda3/envs/t2d_pred/lib/python3.5/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(unit_forget_bias=True, recurrent_initializer="uniform", kernel_initializer="uniform", recurrent_activation="sigmo

In [28]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

lstm_callback = model.fit(x_train, y_train, batch_size=batch_size,
                    verbose=2, validation_data=(x_val, y_val),
                    epochs=epochs,
                    callbacks = [custom_callback(
                            training_data=(x_train, y_train),
                            validation_data=(x_val, y_val))])

score = model.evaluate(x_val, y_val, verbose=0)
    
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

pred_test = model.predict(x_test)
roc_score_test = roc_auc_score(y_test, pred_test)

Train on 4925 samples, validate on 2427 samples
Epoch 1/20
roc-auc: 0.7719 - roc-auc_val: 0.7647                                                                                                    
104s - loss: 1.5272 - acc: 0.2847 - val_loss: 1.3090 - val_acc: 0.3502
Epoch 2/20
roc-auc: 0.797 - roc-auc_val: 0.79                                                                                                    
79s - loss: 1.2778 - acc: 0.3498 - val_loss: 1.2622 - val_acc: 0.3506
Epoch 3/20
roc-auc: 0.7988 - roc-auc_val: 0.7954                                                                                                    
76s - loss: 1.2592 - acc: 0.3553 - val_loss: 1.2649 - val_acc: 0.3502
Epoch 4/20
roc-auc: 0.8157 - roc-auc_val: 0.8114                                                                                                    
80s - loss: 1.2530 - acc: 0.3553 - val_loss: 1.2457 - val_acc: 0.3523
Epoch 5/20
roc-auc: 0.8285 - roc-auc_val: 0.8253                              

In [29]:
roc_score_test

0.85674422515193516